# Programming a neural network layer

[Keras](https://keras.io) is a high-level deep-learning framework building on top of [TensorFlow](https://www.tensorflow.org). These frameworks follow the _symbol-to-symbol derivatives_ approach, i.e. automatically derive a computational graph to calculate derivatives. You just need to declare your inputs as TensorFlow variables and use TensorFlow operations on them to compute the forward pass.  

## Task 6.1

Work through the [Keras tutorial on custom layers](https://keras.io/guides/making_new_layers_and_models_via_subclassing) to learn how to create your own neural network layer.  
Create a custom Keras layer that computes Gaussian basis functions, i.e. a layer that maps an input vector $\mathbf x \in \mathbb R^n$ onto an output vector $\mathbf y = f(\mathbf x) \in \mathbb R^m$ as follows:
\begin{align}
  f: \mathbf x \in \mathbb R^n \mapsto \left[w_i \exp\left(-\frac{\|\mathbf x - \boldsymbol\mu_i\|^2}{\sigma_i^2}\right)\right]_{i=1..m} \in \mathbb R^m
\end{align}

Instead of projecting an input $\mathbf x$ onto a weight vector $\mathbf w$ as the standard neuron function $f(\mathbf x) = \sigma(\mathbf w \cdot \mathbf x + b)$ does, the Gaussian basis function becomes active (with weight $w_i$) for all inputs $\mathbf x$ close to a prototype $\boldsymbol \mu_i$. This activation quickly decays with increasing distance of $\mathbf x$ to $\boldsymbol \mu_i$. The parameter $\sigma_i$ controls the width of the Gaussian, i.e. the size of the active region.

For efficient tensor-based operations you need to correctly _broadcast_ the tensors for the difference operation: TensorFlow will pass an input matrix of shape `(batch size, input dim)` for $\mathbf X$, while you will have a matrix of centers $\boldsymbol \mu$ of shape `(input dim, #units)`. To correctly [broadcast](https://numpy.org/doc/stable/user/basics.broadcasting.html) them together, you will need Keras' [`expand_dims()`](https://www.tensorflow.org/api_docs/python/tf/keras/backend/expand_dims) function to extend $\mathbf X$'s shape to `(batch size, input dim, 1)`:

In [48]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

class RBFLayer(keras.layers.Layer):
    def __init__(self, output_dim, initializer=None, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.initializer = initializer

    def build(self, input_shape):
        self.centers = self.add_weight(
            name='centers',
            shape=(self.output_dim, input_shape[1]),
            initializer=self.initializer,
            trainable=True)
        self.sigmas = self.add_weight(
            name='sigmas',
            shape=(self.output_dim,),
            initializer='ones',
            trainable=True)

    def call(self, inputs):
        inputs = K.expand_dims(inputs, 1)
        diff = inputs - self.centers
        norm = K.sum(K.square(diff), axis=-1)
        norm = K.transpose(norm)
        return K.exp(-norm / (2 * K.square(self.sigmas)))

# Example usage:
X = tf.ones((3, 5))  # input tensor X with batch dimension 3 and data dim N=5
mu = tf.ones((5, 2))  # tensor mu with data dim N=5 and 2 units
diffs = K.expand_dims(X, axis=-1) - mu  # diffs tensor: 3 x 5 x 2
print(X.shape, mu.shape, diffs.shape)

(3, 5) (5, 2) (3, 5, 2)


## Task 6.2

Compare the performance of such a Gaussian basis function layer with that of a standard [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layer on the MNIST dataset.  
Hint: Utilize existing tutorials on setting up your first MNIST MLP with Keras, e.g. https://www.tensorflow.org/guide/keras/train_and_evaluate.

To achieve decent performance, you want to:
- Initialize the centers $\boldsymbol \mu_i$ from random data samples $\mathbf x$ (create a custom [initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/Initializer))
- Initialize $\sigma_i$ to the typical in-class distance between data points.  
  Use [`scipy.spatial.distance_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html) to compute this statistics on a random selection of your input data.  
  (Doing it on the full dataset will probably exhaust your memory.)
- Initialize $w_i = 1$

Questions:
- How many parameters each of those networks have?
- Which network trains faster / easier?

In [46]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from scipy.spatial import distance_matrix
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocessing the data
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Get random set of 1000 mnist images
randomset = x_train[np.random.choice(x_train.shape[0], 1000, replace=False)]
distances = distance_matrix(randomset, randomset)

# Initialize to typical in-class distances

sigma = np.mean(distances)

class MuInitializer(keras.initializers.Initializer):
    def __init__(self, data):
        self.data = data

    def __call__(self, shape, dtype=None):
        centers = self.data[np.random.choice(len(self.data), size=shape[0])]
        return tf.convert_to_tensor(centers, dtype=dtype)

# The Dense Layer model

inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1", kernel_initializer="glorot_uniform")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2", kernel_initializer="glorot_uniform")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

results = model.evaluate(x_test, y_test, batch_size=128)

print("test loss, test acc:", results)

predictions = model.predict(x_test)
predicted_labels = tf.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predicted_labels)

print(f"Accuracy score:{accuracy}")

Epoch 1/5
750/750 [==============================] - 1s 1ms/step - loss: 0.3599 - sparse_categorical_accuracy: 0.8975 - val_loss: 0.1889 - val_sparse_categorical_accuracy: 0.9473
Epoch 2/5
750/750 [==============================] - 1s 906us/step - loss: 0.1735 - sparse_categorical_accuracy: 0.9485 - val_loss: 0.1578 - val_sparse_categorical_accuracy: 0.9521
Epoch 3/5
750/750 [==============================] - 1s 860us/step - loss: 0.1247 - sparse_categorical_accuracy: 0.9625 - val_loss: 0.1313 - val_sparse_categorical_accuracy: 0.9607
Epoch 4/5
750/750 [==============================] - 1s 884us/step - loss: 0.0984 - sparse_categorical_accuracy: 0.9705 - val_loss: 0.1104 - val_sparse_categorical_accuracy: 0.9670
Epoch 5/5
79/79 [==============================] - 0s 528us/step - loss: 0.1022 - sparse_categorical_accuracy: 0.9700
test loss, test acc: [0.10224821418523788, 0.9700000286102295]
313/313 [==============================] - 0s 363us/step
Accuracy score:0.97


In [49]:
# The Gaussian RBF Layer model

inputs = keras.Input(shape=(784,), name="digits")
x = RBFLayer(64, initializer=MuInitializer(randomset))(inputs)
x = RBFLayer(64, initializer=MuInitializer(randomset))(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.2)

results = model.evaluate(x_test, y_test, batch_size=128)

print("test loss, test acc:", results)

predictions = model.predict(x_test)
predicted_labels = tf.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, predicted_labels)

print(f"Accuracy score:{accuracy}")

ValueError: In this `tf.Variable` creation, the initial value's shape ((64, 784)) is not compatible with the explicitly supplied `shape` argument ((64, 64)).